# Simple Linear Regression

In this notebook we will use data on house sales in King County to predict house prices using simple (one input) linear regression. You will:
* Use the pandas library to load existing dataset
* Write a function to compute the Simple Linear Regression weights using the closed form solution
* Write a function to make predictions of the output given the input feature
* Turn the regression around to predict the input given the output
* Compare two different models for predicting house prices

In this notebook you will be provided with some already complete code as well as some code that you should complete yourself in order to answer quiz questions. The code we provide to complte is optional and is there to assist you with solving the problems but feel free to ignore the helper code and write your own.

## Import the libraries

In [1]:
import sklearn, pandas
import numpy as np

## Load house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
full_data = pandas.read_csv("kc_house_data.csv", index_col=0)
full_data.head(5)

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
id,,,,,,,,,,,,,,,,,,,,
7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


## View the read data

We should split the data into train / test set. To maintain runtime consistency, we will need a seed value (default = 0)

In [3]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(full_data, train_size=0.8, test_size=0.2, random_state=0)

## Useful Pandas functions

In order to make use of the closed form solution as well as taking advantage of pandas's built-in capacity, we will do some simple exercise. In particular:
* Computing the sum of a column
* Computing the arithmetic average (mean) of the columns
* multiplying and updating a column with a constant
* multiplying a column with another column's value

In [4]:
# Let's compute the mean of the House Prices in King County in 2 different ways.
prices = full_data['price']

# recall that the arithmetic average (the mean) is the sum of the prices divided by the total number of houses:
avg_price_auto = prices.mean()
avg_price_manual = sum(prices.values) / float(len(prices))
print("average price (function): %.2f" % (avg_price_auto))
print("average price (manual):  %.2f" % (avg_price_manual))

average price (function): 540088.14
average price (manual):  540088.14


In [5]:
# if we want to multiply every price by 0.5 it's a simple as:
half_prices = 0.5*prices
# Let's compute the sum of squares of price. We can multiply two SArrays of the same length elementwise also with *
prices_squared = prices*prices
sum_prices_squared = prices_squared.sum() # price_squared is an SArray of the squares and we want to add them up.
print("the sum of price squared is: %.2E" % (sum_prices_squared))

the sum of price squared is: 9.22E+15


Aside: The python notation x.xxe+yy means x.xx \* 10^(yy). e.g 100 = 10^2 = 1*10^2 = 1e2

Replace the exponential notation (%.2E) with the float notation (%.4f) and see the display difference

## Build a generic simple linear regression function 

Armed with these functions we can use the closed form solution found from lecture to compute the slope and intercept for a simple linear regression on observations stored as DataFrame columns: input_feature, output.

Complete the following function (or write your own) to compute the simple linear regression slope and intercept:

In [25]:
def simple_linear_regression(input_feature, output):
    # compute the sum of input_feature and output
    sum_input_feature = input_feature.sum()
    sum_output = output.sum()
    # compute the product of the output and the input_feature and its mean
    product_output_input = (output*input_feature).sum()
    product_mean = product_output_input / float(len(input_feature))
    # compute the squared value of the input_feature and its mean
    squared_input_feature = (input_feature*input_feature).sum()
    squared_mean = squared_input_feature / float(len(input_feature))
    # use the formula for the numerator and denominator
    numerator = product_mean - (sum_input_feature/float(len(input_feature)))*(sum_output/float(len(input_feature)))
    denominator = squared_mean - (sum_input_feature/float(len(input_feature)))*(sum_input_feature/float(len(input_feature)))
    # use the formula for the slope
    slope = numerator/denominator
    # use the formula for the intercept
    intercept = (sum_output / float(len(input_feature))) - slope*(sum_input_feature/float(len(input_feature)))
    return (intercept, slope)

We can test that our function works by passing it something where we know the answer. In particular we can generate a feature and then put the output exactly on a line: output = 1 + 1\*input_feature then we know both our slope and intercept should be 1

In [26]:
mock_feature = np.array(range(5))
mock_output = 1 + 1*mock_feature
(mock_intercept, mock_slope) =  simple_linear_regression(mock_feature, mock_output)
print("Intercept: %.2f" % (mock_intercept))
print("Slope: %.2f" % (mock_slope))

Intercept: 1.00
Slope: 1.00


Now that we know it works let's build a regression model for predicting price based on sqft_living. Rembember that we train on train_data!

In [27]:
sqft_features = train_data['sqft_living'].values
sqft_labels = train_data['price'].values
sqft_intercept, sqft_slope = simple_linear_regression(sqft_features, sqft_labels)

print("Intercept: %.2f" % (sqft_intercept))
print("Slope: %.2f" % (sqft_slope))

Intercept: -48257.06
Slope: 283.97


# Predicting Values

Now that we have the model parameters: intercept & slope we can make predictions. Using numpy, it's easy to multiply a numpy array by a constant and add a constant value. Complete the following function to return the predicted output given the input_feature, slope and intercept:

In [31]:
def get_regression_predictions(input_feature, intercept, slope):
    predicted_values = intercept + slope * np.array(input_feature)
    return predicted_values

Now that we can calculate a prediction given the slope and intercept let's make a prediction. Use (or alter) the following to find out the estimated price for a house with 2000 squarefeet according to the squarefeet model we estimated above.

**Quiz Question: Using your Slope and Intercept from (4), What is the predicted price for a house with 2000 sqft?**

In [32]:
my_house_sqft = 2000
estimated_price = get_regression_predictions(my_house_sqft,sqft_intercept,sqft_slope)
print("The estimated price for a house with %d squarefeet is $%.4f" % (my_house_sqft, estimated_price))

The estimated price for a house with 2000 squarefeet is $519680.0507


# Residual Sum of Squares

Now that we have a model and can make predictions let's evaluate our model using Residual Sum of Squares (RSS). Recall that RSS is the sum of the squares of the residuals and the residuals is just a fancy word for the difference between the predicted output and the true output. 

Complete the following (or write your own) function to compute the RSS of a simple linear regression model given the input_feature, output, intercept and slope:

In [38]:
def get_residual_sum_of_squares(input_feature, output, intercept, slope):
    # First get the predictions
    predicted_values = get_regression_predictions(input_feature,intercept,slope)
    # then compute the residuals (since we are squaring it doesn't matter which order you subtract)
    residuals = output - predicted_values
    # square the residuals and add them up
    RSS = (residuals*residuals).sum()
    return(RSS)

Let's test our `get_residual_sum_of_squares` function by applying it to the test model where the data lie exactly on a line. Since they lie exactly on a line the residual sum of squares should be zero!

In [40]:
print("%.2f" % get_residual_sum_of_squares(mock_feature, mock_output, mock_intercept, mock_slope)) # should be 0.0

0.00


Now use your function to calculate the RSS on training data from the squarefeet model calculated above.

**Quiz Question: According to this function and the slope and intercept from the squarefeet model, what is the RSS for the simple linear regression using squarefeet to predict prices on TRAINING data?**

In [41]:
train_features = train_data['sqft_living'].values
train_labels = train_data['price']
rss_prices_on_sqft = get_residual_sum_of_squares(train_features,train_labels,sqft_intercept,sqft_slope)
print('The RSS of predicting Prices based on Square Feet is : %.2E' % (rss_prices_on_sqft))

The RSS of predicting Prices based on Square Feet is : 1.21E+15


# Predict the squarefeet given price

What if we want to predict the squarefoot given the price? Since we have an equation $y = a + b*x$ we can solve the function for x. So that if we have the intercept (a) and the slope (b) and the price (y) we can solve for the estimated squarefeet (x).

Complete the following function to compute the inverse regression estimate, i.e. predict the input_feature given the output.

In [42]:
def inverse_regression_predictions(output, intercept, slope):
    estimated_feature = (output - intercept)/slope
    return estimated_feature

Now that we have a function to compute the squarefeet given the price from our simple regression model let's see how big we might expect a house that costs $800,000 to be.

**Quiz Question: According to this function and the regression slope and intercept from (3) what is the estimated square-feet for a house costing $800,000?**

In [44]:
# instant result, just add your parameters in the function below
my_house_price = 800000
estimated_squarefeet = inverse_regression_predictions(my_house_price,sqft_intercept,sqft_slope)
print("The estimated squarefeet for a house worth $%.2f is %d" % (my_house_price, estimated_squarefeet))

The estimated squarefeet for a house worth $800000.00 is 2987


# New Model: estimate prices from bedrooms

We have made one model for predicting house prices using squarefeet, but there are many other features in the sales data. 
Use your simple linear regression function to estimate the regression parameters from predicting Prices based on number of bedrooms. Use the training data!

In [46]:
room_features = train_data['bedrooms'].values
room_labels = train_data['price'].values
room_intercept, room_slope = simple_linear_regression(room_features, room_labels)

print("Intercept: %.2f" % (room_intercept))
print("Slope: %.2f" % (room_slope))

Intercept: 126751.85
Slope: 123535.46


# Test your Linear Regression Algorithm

Now we have two models for predicting the price of a house. How do we know which one is better? Calculate the RSS on the TEST data (remember this data wasn't involved in learning the model). Compute the RSS from predicting prices using bedrooms and from predicting prices using squarefeet.

**Quiz Question: Which model (square feet or bedrooms) has lowest RSS on TEST data? Think about why this might be the case.**

In [47]:
# Compute RSS when using bedrooms on TEST data. Handholding level: high.
bdrm_test_features = test_data['bedrooms'].values
bdrm_test_labels = test_data['price'].values
rss_prices_on_bedroom = get_residual_sum_of_squares(bdrm_test_features,bdrm_test_labels,room_intercept,room_slope)
print("Bedroom regressions result: %.2E" % rss_prices_on_bedroom)

Bedroom regressions result: 4.73E+14


In [48]:
# Compute RSS when using squarefeet on TEST data. Handholding level: not as high
sqft_test_features = test_data['sqft_living'].values
sqft_test_labels = test_data['price'].values
rss_prices_on_sqft_living = get_residual_sum_of_squares(sqft_test_features,sqft_test_labels,sqft_intercept,sqft_slope)
print('Sqft_living regressions result: %.2E'% rss_prices_on_sqft_living)

Sqft_living regressions result: 2.68E+14
